In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/Test/"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
%matplotlib inline

In [ ]:
import pandas as pd
test = pd.read_csv("/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/BERT-base/train_set.csv")
test = test.dropna()
df = pd.read_csv('/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/BERT-base/test_set.csv')
df = df.dropna()
print(df.head())
print(test.head())

In [ ]:
data = df.review.tolist()
label = df.label.tolist()
data_test = test.review.tolist()
label_test = test.label.tolist()
print(len(data))
print(len(data_test))
data_n = []
for i in data:
  i = i.replace('_','')
  data_n.append(i)
test_v = []
for i in data_test:
  i = i.replace('_','')
  test_v.append(i)

In [ ]:
import pickle
with open('/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/Test/glove.model', 'rb') as pickle_file:
    embeddings_index = pickle.load(pickle_file)

In [ ]:
maxlen = 300

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer  = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(data_n)

word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))

sequences =  tokenizer.texts_to_sequences(data_n)
sequences_test =  tokenizer.texts_to_sequences(test_v)
# maxlen = max([len(i.split()) for i in data_n])
# print(maxlen)
maxlen = 300
data1 = pad_sequences(sequences, maxlen=maxlen)
test1 = pad_sequences(sequences_test, maxlen=maxlen)

le = LabelEncoder()

label = le.fit_transform(label)
labels = label.reshape(-1,1)
labels_test = le.fit_transform(label_test)
labels_test = labels_test.reshape(-1,1)
labels = to_categorical(np.asarray(label))
labels_test = to_categorical(np.asarray(label_test))
print(data1.shape)
print('Shape of label tensor:', labels.shape)

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(data1,label,test_size=0.5, random_state=48)

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
print(300)
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Input, Flatten, SpatialDropout1D
from tensorflow.keras.layers import Conv1D,MaxPool1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda
from tensorflow.keras.models import Model

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True)

sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
bert_out = SpatialDropout1D(0.5)(embedded_sequences)
filter_lengths = [3, 4, 5]
conv_layers = []
for filter_length in filter_lengths:
    conv_layer = Conv1D(filters=100, kernel_size=filter_length, padding='valid',
                        strides=1, activation='relu')(bert_out)
    maxpooling = MaxPool1D(pool_size=128 - filter_length + 1)(conv_layer)
    flatten = Flatten()(maxpooling)
    conv_layers.append(flatten)
    
sentence_embed = Concatenate()(conv_layers)

dense_layer = Dense(128, activation='relu')(sentence_embed)
preds = Dense(1, activation='sigmoid')(dense_layer)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.0003),
              metrics=['acc'])
model.summary()


In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(path+'/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/Test/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]


In [ ]:
history = model.fit(x_train,y_train,epochs=10,batch_size=16,verbose = 1,validation_data=[x_val,y_val],callbacks=callbacks_list)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
def convert_to_sequences(texts):
  sequences =  tokenizer.texts_to_sequences(texts)
  return pad_sequences(sequences, maxlen=maxlen)

In [ ]:
from keras.models import load_model
model = load_model( '/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/Test/model.h5')

In [ ]:
from sklearn import metrics
acc = model.predict(convert_to_sequences(test_v))
accr = np.round(acc)
print(metrics.f1_score(label_test, accr))